## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

| Food  | Cost | Availability |
|-------|------|--------------|
| Bread | 0.1  | 4            |
| Milk  | 0.5  | 3            |
| Eggs  | 0.12 | 1            |
| Meat  | 0.9  | 2            |
| Cake  | 1.3  | 2            |

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

**Relevant sets:**

$I$ = set of types of fiids \\
$J$ = set of types of nutrients

$I$ = {Bread, MIlk, Eggs, ...} when we refer ti elements we use $i$ \\
$J$ = {Calories, Proteins, Calcium} when we refer ti elements we use $j$

**Decision variables:**

$x_i$ amount of food type $i \in I$

**Parameters:**

From Table 1:

$c_i$ = cost of food type $i \in I$ \\
$q_i$ = available quantity iof food type $i \in I$ \\

From Table 2:

$b_j$ = minimum quantity of nutrient of type $j \in J$ \\

From Table 3:

$a_{ij}$ = the amount of nutrient $j \in J$ for the food type $i \in I$ \\

**Constrains**

MInimum quantity of nutrients in the solutions (3 constrains, one for each nutrient)

$$ \sum_{i \in I} a_{ij} x_{i} \geq b_j , \ \ \ \forall j \in J $$

Availability of food types:

$$ x_i \leq q_i, \ \ \ \forall i \in I $$

Nature of decision variables:

$$ x_i \geq 0 $$

**Objective function:**

$$ \min \sum_{i \in I} c_i x_i $$

$$ \min 0.1x_1 + 0.5x_2 + 0.12x_3 + 0.9x_4 + 1.3x_5 $$



In [1]:
# When using Colab, make sure you run this instruction beforehand
#!pip install mip

In [2]:
# We need to import the package mip
import mip

In [3]:
# Food
I = ['Bread', 'Milk', 'Eggs', 'Meat', 'Cake']

# Nutrients
J = ['Calories', 'Proteins', 'Calcium']

# Cost in Euro per hg of food
c = {'Bread':0.1, 'Milk':0.5, 'Eggs':0.12, 'Meat':0.9, 'Cake':1.3}

# Availability per hg of food
q = {'Bread':4, 'Milk':3, 'Eggs':1, 'Meat':2, 'Cake':2}

# Minimum nutrients
b = {'Calories':600, 'Proteins':50, 'Calcium':0.7}

# Nutrients per hf of food
a = {('Bread','Calories'):30,('Milk','Calories'):50,('Eggs','Calories'):150,('Meat','Calories'):180,('Cake','Calories'):400,
('Bread','Proteins'):5,('Milk','Proteins'):15,('Eggs','Proteins'):30,('Meat','Proteins'):90,('Cake','Proteins'):70,
('Bread','Calcium'):0.02,('Milk','Calcium'):0.15,('Eggs','Calcium'):0.05,('Meat','Calcium'):0.08,('Cake','Calcium'):0.01}

In [4]:
# Print

print('\nCost in Euro per hg of food')
for i,j in c.items():
  print(i,j)

print('\nAvailability per hg of food')
for i,j in q.items():
  print(i,j)

print('\nMinimum nutrients')
for i,j in b.items():
  print(i,j)



Cost in Euro per hg of food
Bread 0.1
Milk 0.5
Eggs 0.12
Meat 0.9
Cake 1.3

Availability per hg of food
Bread 4
Milk 3
Eggs 1
Meat 2
Cake 2

Minum nutrients
Calories 600
Proteins 50
Calcium 0.7


Now we create an empty model and add the variables

In [5]:
# Define a model
model = mip.Model()

# add_var()

# One way: one by one
# x1 = model.add_var(var_type = mip.CONTINUOUS) # by default x1 >=0
# x2 = model.add_var()

# Second way : using llist comprehension
x = [model.add_var(name = i,lb=0) for i in I]

In [6]:
x

Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [7]:
# Define the objective function

# the function xsum() receives as input a lit
#outputs the sum for the model to read it
model.objective = mip.minimize(mip.xsum([c[food]*x[i] for i, food in enumerate(I)]))



The constraints can be generated in loops:

In [8]:
# CONSTRAINTS

# Availability constraint
for i,food in enumerate(I):
  model.add_constr(x[i] <= q[food])

# Minimum nutrients
for j in J:
  model.add_constr(mip.xsum([a[food,j]*x[i] for i, food in enumerate(I)])>= b[j])

The model is complete. Let's solve it and print the optimal solution.

In [9]:
# Optimizing command
model.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming problem using Dual Simplex



<OptimizationStatus.OPTIMAL: 0>

In [10]:
# Optimal objective function value
model.objective.x

3.37

In [11]:
# Printing the variables values
for i in model.vars:
  print(i.name,"\t=\t", i.x)

Bread 	=	 3.9999999999999996
Milk 	=	 3.0
Eggs 	=	 1.0
Meat 	=	 1.5000000000000002
Cake 	=	 0.0


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


In [12]:
import mip

# Set of oil types
I = [1, 2, 3, 4]

# Set of gasoline types
J = ["A", "B", "C"]

# Unit cost for oil of type i
c = [9, 7, 12, 6]

# Availability of oil type i
a = [5000, 2400, 4000, 1500]

# Price of gasoline of type j
r = [12, 18, 10]

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1
q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0
q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

Now we create an empty model and add the variables

In [13]:
# Define a model
model2 = mip.Model()

# Define variables
x = [[model2.add_var(name=str(i) + j) for j in J] for i in I]
y = [model2.add_var(name="N" + str(j)) for j in J]

# default --> variabili reali, lower bound = 0 e upper bound = +INF --> edita a seconda di ciò che ti serve

In [14]:
# Define the objective function
model2.objective = mip.maximize(
  mip.xsum([r[j] * y[j] for j, gas in enumerate(J)])
  -
  mip.xsum([c[i] * mip.xsum([x[i][j] for j in range(len(J))]) for i in range(len(I))])
)

In [15]:
# CONSTRAINTS
# Availability constraint
for i in range(len(I)):
  model2.add_constr(mip.xsum([x[i][j] for j in range(len(J))]) <= a[i])

# Conservation constraint
for j in range(len(J)):
  model2.add_constr(mip.xsum([x[i][j] for i in range(len(I))]) == y[j])

# Maximum quantity
for i, oil in enumerate(I):
  for j, gas in enumerate(J):
    model2.add_constr(x[i][j] <= q_max[(str(oil), gas)] * y[j])

# Minimum quantity
for i, oil in enumerate(I):
  for j, gas in enumerate(J):
    model2.add_constr(x[i][j] >= q_min[(str(oil), gas)] * y[j])

The model is complete. Let's solve it and print the optimal solution.

In [16]:
# Optimizing command
model2.optimize()

Coin0506I Presolve 2 (-6) rows, 5 (0) columns and 10 (-10) elements
Clp0000I Optimal - objective value 3.37
Coin0511I After Postsolve, objective 3.37, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 3.37 - 1 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex



<OptimizationStatus.OPTIMAL: 0>

In [17]:
# Optimal objective function value
model2.objective.x

72000.00000000001

In [18]:
# Printing the variables values
for i in model2.vars:
  print(i.name)
  print(i.x)
  print('-----')

1A
5000.0
-----
1B
0.0
-----
1C
0.0
-----
2A
0.0
-----
2B
2400.0
-----
2C
0.0
-----
3A
1900.0
-----
3B
2100.0
-----
3C
0.0
-----
4A
0.0
-----
4B
1500.0
-----
4C
0.0
-----
NA
6900.000000000001
-----
NB
6000.0
-----
NC
0.0
-----


In [19]:
# Ecco come risolvere con MIP un problema di linear programming --> potresti risolverlo a mano, molto più lento